# What-if scenarios

In this stage of the analysis, we want to find how changes to the parameters of our model affect the simulated data.

Some areas of interest are:

- How would server utilisation (i.e. resource consumption) be affected by an increase in overall patient arrivals?
- How is the system affected by certain types of patients (e.g. short-stay, low-impact) arriving less frequently?
- What are the sensitivities of mean system times and server utilisation based on a change in $c$?

In [1]:
import glob
import itertools as it
import time

from ciw.dists import Exponential
from dask.diagnostics import ProgressBar
from scipy import stats

import ciw
import dask
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline
plt.style.use("seaborn-colorblind")

In [2]:
copd = pd.read_csv(
    "../data/clusters/copd_clustered.csv", parse_dates=["admission_date", "discharge_date"]
).dropna(subset=["cluster"])

copd["cluster"] = copd["cluster"].astype(int)
n_clusters = copd["cluster"].nunique()

In [3]:
with open("../data/wasserstein/best/params.txt", "r") as f:
    strings = f.read().split(" ")
    props, num_servers = list(map(float, strings[:-2])), int(strings[-2])

props, num_servers

([0.95, 1.0, 1.0, 0.5], 40)

## Change in overall patient arrivals

In [ ]:
lambda_results = pd.concat(
    (pd.read_csv(filename) for filename in glob.iglob("../data/lambda_scaling/*.csv"))
)

In [ ]:
lambda_originals = lambda_results[lambda_results["lambda_coeff"].round(2) == 1]
lambda_mean_utilisation = lambda_originals["utilisation"].mean()
lambda_mean_system_time = lambda_originals["system_time"].mean()

lambda_results["relative_utilisation"] = lambda_results["utilisation"] / lambda_mean_utilisation
lambda_results["relative_system_time"] = lambda_results["system_time"] / lambda_mean_system_time

In [ ]:
lambda_utilisations = lambda_results.groupby("lambda_coeff")["relative_utilisation"]

lower_utilisation = lambda_utilisations.quantile(0.025)
mean_utilisation = lambda_utilisations.mean()
upper_utilisation = lambda_utilisations.quantile(0.975)

In [ ]:
fig, ax = plt.subplots(dpi=200)

ax.plot(mean_utilisation, label="Mean")
ax.fill_between(
    lower_utilisation.index, lower_utilisation, upper_utilisation, alpha=0.25, label="95% interval"
)

ax.legend()
ax.set(
    xlabel="$\lambda$ scaling factor",
    ylabel="Relative server utilisation",
)

plt.savefig("../img/lambda_utilisation.pdf", transparent=True)

In [ ]:
lambda_times = lambda_results.groupby("lambda_coeff")["relative_system_time"]

lower_times = lambda_times.quantile(0.025)
mean_times = lambda_times.mean()
upper_times = lambda_times.quantile(0.975)

In [ ]:
fig, ax = plt.subplots(dpi=200)

ax.plot(mean_times, label="Mean")
ax.fill_between(
    lower_times.index, lower_times, upper_times, alpha=0.25, label="95% interval"
)

ax.legend()
ax.set(
    xlabel="$\lambda$ scaling factor",
    ylabel="Relative system time (logscale)",
    yscale="log",
)

plt.savefig("../img/lambda_time.pdf", transparent=True)

## Sensitivity analysis for $c$

In [ ]:
server_results = pd.concat(
    (pd.read_csv(filename) for filename in glob.iglob("../data/num_servers_change/*.csv"))
)

server_results["relative_num_servers"] = server_results["num_servers"] / num_servers

In [ ]:
server_originals = server_results[server_results["relative_num_servers"] == 1]
server_mean_utilisation = server_originals["utilisation"].mean()
server_mean_system_time = server_originals["system_time"].mean()

server_results["relative_utilisation"] = server_results["utilisation"] / server_mean_utilisation
server_results["relative_system_time"] = server_results["system_time"] / server_mean_system_time

In [ ]:
server_utilisations = server_results.groupby("relative_num_servers")["relative_utilisation"]

lower_utilisation = server_utilisations.quantile(0.025)
mean_utilisation = server_utilisations.mean()
upper_utilisation = server_utilisations.quantile(0.975)

In [ ]:
fig, ax = plt.subplots(dpi=200)

ax.plot(mean_utilisation, label="Mean")
ax.fill_between(
    lower_utilisation.index, lower_utilisation, upper_utilisation, alpha=0.25, label="95% interval"
)

ax.legend()
ax.set(
    xlabel="Relative no. of servers",
    ylabel="Relative server utilisation",
)

plt.savefig("../img/servers_utilisation.pdf", transparent=True)

In [ ]:
server_times = server_results.groupby("relative_num_servers")["relative_system_time"]

lower_times = server_times.quantile(0.025)
mean_times = server_times.mean()
upper_times = server_times.quantile(0.975)

In [ ]:
fig, ax = plt.subplots(dpi=200)

ax.plot(mean_times, label="Mean")
ax.fill_between(
    lower_times.index, lower_times, upper_times, alpha=0.25, label="95% interval"
)

ax.legend()
ax.set(
    xlabel="Relative no. of servers",
    ylabel="Relative system time (logscale)",
    yscale="log",
)

plt.savefig("../img/servers_time.pdf", transparent=True)

## Transferring patients between clusters

In [ ]:
moving_results = pd.concat(
    [pd.read_csv(filename) for filename in glob.iglob("../data/moving_clusters/*.csv")]
)

In [ ]:
def get_plotting_data(data, column):

    data = data.copy()
    original_mean = data[data["prop_to_move"] == 0][column].mean()
    data[f"relative_{column}"] = data[column] / original_mean

    values = data.groupby("prop_to_move")[f"relative_{column}"]

    lower = values.quantile(0.025)
    mean = values.mean()
    upper = values.quantile(0.975)

    return lower, mean, upper


def make_split_plot(column, ylabel_text, path=None, yscale="linear"):

    fig, axes = plt.subplots(
        nrows=n_clusters,
        ncols=n_clusters,
        sharex=True,
        sharey=True,
        figsize=(7, 7),
        dpi=300,
    )

    left, width = .25, .5
    bottom, height = .25, .5
    right = left + width
    top = bottom + height

    ylabel = fig.text(
        0, 0.5*(bottom+top), ylabel_text, ha="right", va="center", rotation="vertical"
    )

    xlabel = fig.text(
        0.5*(left+right), -0.01, "Transfer proportion", ha="center", va="bottom", rotation="horizontal"
    )

    for origin, destination in it.product(range(n_clusters), repeat=2):

        ax = axes[origin, destination]
        ax.set_yscale(yscale)

        if origin != destination:
            data = moving_results[
                (moving_results["origin"] == origin)
                & (moving_results["destination"] == destination)
            ]

            lower, mean, upper = get_plotting_data(data, column)

            ax.plot(mean)
            ax.fill_between(lower.index, lower, upper, alpha=0.25)
            ax.set_title(f"{origin} to {destination}")

    plt.tight_layout()
    if path is not None:
        plt.savefig(
            path,
            bbox_extra_artists=(xlabel, ylabel),
            bbox_inches="tight",
            transparent=True,
        )

In [ ]:
make_split_plot(
    column="utilisation",
    ylabel_text="Relative server utilisation",
    path="../img/moving_utilisation.pdf",
)

In [ ]:
make_split_plot(
    column="system_time",
    ylabel_text="Relative system time (logscale)",
    path="../img/moving_time.pdf",
    yscale="log",
)

In [ ]:
fig, axes = plt.subplots(nrows=n_clusters, ncols=n_clusters, sharex=True, sharey=True, figsize=(7, 7), dpi=300)

left, width = .25, .5
bottom, height = .25, .5
right = left + width
top = bottom + height

ylabel = fig.text(
    0, 0.5*(bottom+top), "Relative system time", ha="right", va="center", rotation="vertical"
)

xlabel = fig.text(
    0.5*(left+right), -0.01, "Transfer proportion", ha="center", va="bottom", rotation="horizontal"
)

for origin, destination in it.product(range(n_clusters), repeat=2):

    ax = axes[origin, destination]
    ax.set_yscale("log")

    if origin != destination:
        data = moving_results[
            (moving_results["origin"] == origin)
            & (moving_results["destination"] == destination)
        ]

        lines, labels = [], []
        for label, subdata in data.groupby("customer_class"):

            _, mean, _ = get_plotting_data(subdata, "system_time")

            line = ax.plot(mean)
            lines.append(line[0])
            labels.append(label)

        ax.set_title(f"{origin} to {destination}")

legend = ax.legend(
    lines,
    labels,
    ncol=n_clusters,
    loc="upper center",
    bbox_to_anchor=(0.5, 1.08),
    prop={'size': 12},
    title="Cluster",
)

plt.tight_layout()
plt.savefig(
    "../img/moving_time_split.pdf",
    bbox_extra_artists=(xlabel, ylabel, legend),
    bbox_inches="tight",
    transparent=True,
)